<img src="https://gallery.mailchimp.com/f98d5ac0a3fbbdcdda35136ab/images/2002af76-5fd4-4185-9d49-28558b6b8772.png">

# `sg-hdb-resale-bokeh` 
# Part 3: Basic Model Deployment
After all that we have done, exporting as well as serialising the model, now, we create a REST API to deploy our predictive model!

First, let us install the `flask-restful` package. The `flask` package, which is already installed in the Azure Notebook's environment, allows us to create such APIs. The `flask-restful` package is an extension to `flask` which allows us to create such APIs in a more readable manner. So first, let us run the cell below.

In [ ]:
# This command let us interact with the environment's terminal/shell
!pip install flask-restful

After installation of said package, we now create a simple script to test out the `flask` packages. Create a Python script on the Azure Notebook project titled `flask-hello-world.py` and copy the contents of the cell below into it.

In [ ]:
"""
DO NOT RUN THIS CELL!
"""

from flask import Flask
from flask_restful import Resource, Api

app = Flask(__name__)
api = Api(app)

class HelloWorld(Resource):
    def get(self):
        return {'hello': 'world'}

api.add_resource(HelloWorld, '/')

if __name__ == '__main__':
    app.run(debug=True)

Now, we use the following command to run the script above.

In [ ]:
!python './flask-hello-world.py'

After running the cell above, we now make a request to the initialised API to interact with it. Open up a terminal through Azure and run the following line:

```bash
curl -X GET http://127.0.0.1:5000/
```

You should be provided with a simple `"hello" : "world"` response. After this observation, make sure to stop running the cell above.

Now, let us create another Python script that would allow us to utilise the exported linear regression model we have previously created and make predictions with it. Create a Python script on the Azure Notebook project titled `sg-hdb-basic-deployment.py` and copy the contents of the cell below into it.

In [ ]:
"""
DO NOT RUN THIS CELL!
"""

from flask import Flask
from flask_restful import reqparse, Api, Resource
import pickle
import numpy as np

app = Flask(__name__)
api = Api(app)

# specify location of pickled model and load it
file_loc_name = '../models/sg_hdb_lm_v1.pkl'
sg_hdb_loaded_model = pickle.load(open(file_loc_name, 'rb'))

# argument parsing
parser = reqparse.RequestParser()
parser.add_argument('query')

class PredictResalePrice(Resource):
    def get(self):

        # use parser and find the user's query
        args = parser.parse_args()
        user_query = args['query']
        
        user_floor_area_sqm = np.array(float(user_query))
        user_floor_area_sqm_reshaped = user_floor_area_sqm.reshape(-1, 1)
        user_resale_price = sg_hdb_loaded_model.predict(user_floor_area_sqm_reshaped)

        # create JSON object
        output = {'prediction': user_resale_price[0]}
        #output = user_resale_price
        
        return output

# Setup the API resource routing here
# Route the URL to the resource
api.add_resource(PredictResalePrice, '/')

if __name__ == "__main__":
    app.run(debug=True)

After creating the script, run the following cell.

In [ ]:
!python './sg-hdb-basic-deployment.py'

After running the cell above, we now make a request to the initialised API to get a prediction. Open up a terminal through Azure and run the following line:

```shell
curl -X GET http://127.0.0.1:5000/ -d query=65
```

An output shall be returned to you, containing the predicted resale price in response to the query you have provided.

As you can see, we have deployed the model to an API server (albeit a locally hosted one) and successfully submitted a query which in turn we get a response!

With that, we have come to the end of the workshop! Thank you for your participation thus far!

<font color="blue"><h1><center>The End</center></h1></font>